In [1]:
import tempfile

import hipscat_import.catalog.run_import as runner
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from astropy.io import fits
from astropy.table import Table
from hipscat_import.catalog.arguments import ImportArguments
from hipscat_import.catalog.file_readers import FitsReader

In [2]:
filename = "/data3/epyc/data3/hipscat/raw/sdss/calibObj-008162-4-star.fits.gz"
chunksize = 50_000

table = Table.read(filename, memmap=True)
# table.remove_columns(multidimensional)

total_rows = len(table)
read_rows = 0

while read_rows < total_rows:
    print(len(table[read_rows : read_rows + chunksize].to_pandas()))
    read_rows += chunksize
    print(read_rows)

ValueError: Cannot convert a table with multidimensional columns to a pandas DataFrame. Offending columns are: ['ROWC', 'COLC', 'M_RR_CC', 'M_RR_CC_PSF', 'FLAGS', 'FLAGS2', 'PSP_STATUS', 'PSF_FWHM', 'EXTINCTION', 'SKYFLUX', 'PSFFLUX', 'PSFFLUX_IVAR', 'FIBERFLUX', 'FIBERFLUX_IVAR', 'FIBER2FLUX', 'FIBER2FLUX_IVAR', 'MODELFLUX', 'MODELFLUX_IVAR', 'CALIB_STATUS', 'NMGYPERCOUNT', 'APERFLUX6', 'CMODELFLUX_CLEAN', 'CMODELFLUX_CLEAN_IVAR', 'CMODELFLUX_CLEAN_VAR', 'CMODELFLUX_CLEAN_CHI2', 'CMODEL_CLEAN_NUSE', 'CMODEL_CLEAN_MJD_MAXDIFF', 'CMODEL_CLEAN_MJD_VAR', 'MODELFLUX_CLEAN', 'MODELFLUX_CLEAN_IVAR', 'MODELFLUX_CLEAN_VAR', 'MODELFLUX_CLEAN_CHI2', 'MODEL_CLEAN_NUSE', 'MODEL_CLEAN_MJD_MAXDIFF', 'MODEL_CLEAN_MJD_VAR', 'PSFFLUX_CLEAN', 'PSFFLUX_CLEAN_IVAR', 'PSFFLUX_CLEAN_VAR', 'PSFFLUX_CLEAN_CHI2', 'PSF_CLEAN_NUSE', 'PSF_CLEAN_MJD_MAXDIFF', 'PSF_CLEAN_MJD_VAR']
One can filter out such columns using:
names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
tbl[names].to_pandas(...)

In [8]:
import glob
import re

files = glob.glob("/data3/epyc/data3/hipscat/raw/sdss/**.fits.gz")
files.sort()

for file in files[:5]:
    match = re.match(r".*(calibObj-.*-star).fits.gz", str(file))
    file_prefix = band = match.group(1)
    print(file_prefix)
    full_name = f"/data3/epyc/data3/hipscat/raw/sdss/parquet{file_prefix}.parquet"
    
    table = Table.read(filename)
    table.write(full_name)

calibObj-000094-1-star


ArrowNotImplementedError: Unsupported numpy type 20

In [89]:
filename = "/data3/epyc/data3/hipscat/raw/sdss/calibObj-008162-4-star.fits.gz"
table = Table.read(filename)
small_table = table[0:4]
small_table.keep_columns(["ID", "RA", "DEC", "SKYFLUX", "PSF_CLEAN_MJD_VAR"])
# small_table.keep_columns(["ID", "RA", "DEC"])
small_table

ID,RA,DEC,SKYFLUX,PSF_CLEAN_MJD_VAR
int16,float64,float64,float32[5],float32[5]
23,331.8884045104337,26.130777678836214,1.0734192 .. 30.953287,-9999.0 .. -9999.0
28,331.8921650779922,26.123544119479924,1.0747597 .. 30.887686,-9999.0 .. -9999.0
31,331.8602951232714,26.22610228791549,0.95386565 .. 30.806057,-9999.0 .. -9999.0
33,331.8730160785075,26.189929422992883,1.0373113 .. 30.852201,-9999.0 .. -9999.0


In [18]:
import numpy as np
flux_data = small_table["SKYFLUX"].data
flux_data[0]
np_data = np.array(flux_data)
np_data

array([[ 1.0734192 ,  1.2279965 ,  4.2127857 ,  9.423098  , 30.953287  ],
       [ 1.0747597 ,  1.2245792 ,  4.219263  ,  9.42205   , 30.887686  ],
       [ 0.95386565,  1.2209423 ,  4.24408   ,  9.466485  , 30.806057  ],
       [ 1.0373113 ,  1.2150102 ,  4.213257  ,  9.398745  , 30.852201  ]],
      dtype=float32)

In [21]:
import pandas as pd
pd_data = pd.DataFrame([small_table["ID"], small_table["RA"], small_table["DEC"], flux_data])
pd_data

,0,1,2,3
0,23,28,31,33
1,331.888405,331.892165,331.860295,331.873016
2,26.130778,26.123544,26.226102,26.189929
3,"[1.0734192, 1.2279965, 4.2127857, 9.423098, 30...","[1.0747597, 1.2245792, 4.219263, 9.42205, 30.8...","[0.95386565, 1.2209423, 4.24408, 9.466485, 30....","[1.0373113, 1.2150102, 4.213257, 9.398745, 30...."


In [95]:
from astropy.table.table import descr
import pyarrow as pa

# new_table = Table()
table_dict = {}

for col in small_table.columns.values():
    descriptor = descr(col)
    col_name = descriptor[0]
    col_shape = descriptor[2]
    if col_shape == (5,):
#         print(col_name)
#         data_t = col.data.T
#         for index, band_char in enumerate('ugriz'):
#             new_table.add_column(data_t[index], name=f"{col_name}_{band_char}")
#         table_dict[col_name] = col.data.tolist()
        pass
    elif col_shape == ():
        table_dict[col_name] = col.data
        
#             new_table.add_column(col)
        
    
# new_table
# new_panda
# new_panda = pd.DataFrame(table_dict)
# new_panda

pa_table = pa.Table.from_pydict(table_dict)

ArrowNotImplementedError: Byte-swapped arrays not supported

In [114]:
%%time

new_table = table.as_array().tolist()

print(new_table)
print(small_table.columns.keys())

new_panda = pd.DataFrame(new_table, columns=table.columns.keys())
new_panda

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CPU times: user 7min 15s, sys: 6.94 s, total: 7min 22s
Wall time: 7min 12s


,RUN,RERUN,CAMCOL,FIELD,ID,OBJC_TYPE,OBJC_FLAGS,OBJC_FLAGS2,OBJC_ROWC,ROWVDEG,...,MODEL_CLEAN_NUSE,MODEL_CLEAN_MJD_MAXDIFF,MODEL_CLEAN_MJD_VAR,PSFFLUX_CLEAN,PSFFLUX_CLEAN_IVAR,PSFFLUX_CLEAN_VAR,PSFFLUX_CLEAN_CHI2,PSF_CLEAN_NUSE,PSF_CLEAN_MJD_MAXDIFF,PSF_CLEAN_MJD_VAR
0,8162,b'301',4,80,23,6,-1845096432,47120,71.752632,-0.007592,...,"[0, 0, 0, 0, 0]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[218.59595, 915.7769, 1480.58, 2052.3486, 2148...","[0.0718015, 0.012233188, 0.39691508, 0.2027919...","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]"
1,8162,b'301',4,80,28,6,-1878126544,47120,83.333847,-0.002670,...,"[0, 0, 0, 0, 0]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[171.86064, 511.53128, 720.16486, 813.867, 884...","[0.112545006, 0.038628615, 0.023287375, 0.3025...","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]"
2,8162,b'301',4,80,31,6,268566528,36880,87.819733,0.000778,...,"[0, 0, 0, 0, 0]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[101.708786, 299.59338, 446.25882, 178.51265, ...","[0.29651058, 0.11170342, 0.05986235, 0.0958653...","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]"
3,8162,b'301',4,80,33,6,268435456,16,97.791443,0.000968,...,"[0, 0, 0, 0, 0]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[49.513783, 211.59148, 363.90958, 428.14236, 4...","[0.9867683, 0.22080493, 0.089316666, 0.0194555...","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]"
4,8162,b'301',4,80,38,6,-1878650864,112656,203.664322,0.007192,...,"[0, 0, 0, 0, 0]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[762.272, 2405.9941, 3597.1545, 4170.511, 4340...","[0.0064542494, 0.4040687, 0.20136414, 0.129388...","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88656,8162,b'301',4,237,1416,6,268435712,65552,1350.132202,0.000000,...,"[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[0.118170395, 0.16061267, 0.48364118, 1.922759...","[29.2821, 213.55005, 64.69467, 26.283098, 1.50...","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]"
88657,8162,b'301',4,237,1418,6,268436224,65552,1374.858887,0.004707,...,"[0, 0, 0, 0, 0]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[0.12945744, 0.08554319, 0.53235054, 1.924389,...","[69.19079, 355.65704, 98.06054, 44.239315, 2.2...","[3.7894282e-05, 0.0010959463, 0.034108058, 0.0...","[0.0012569183, 0.17662013, 1.4073188, 1.211767...","[2, 2, 2, 2, 2]","[2, 2, 2, 2, 2]","[2.0, 2.0, 2.0, 2.0, 2.0]"
88658,8162,b'301',4,237,1420,6,-1879047920,24592,1374.583618,0.004490,...,"[1, 1, 1, 1, 1]","[-9999, -9999, -9999, -9999, -9999]","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-0.34033254, 0.17749062, 0.34574562, 1.552534...","[33.14953, 230.84286, 60.353386, 26.566902, 1....","[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -999

In [115]:
out_filename = "/data3/epyc/data3/hipscat/raw/sdss/parquet_nest/calibObj-008162-4-star.parquet"
# new_panda.to_parquet(out_filename)
# new_table.write(out_filename)
new_panda.to_parquet(out_filename)

In [88]:
pq_table = pd.read_parquet(out_filename)
pq_table

,RUN,RERUN,CAMCOL,FIELD,ID,OBJC_TYPE,OBJC_FLAGS,OBJC_FLAGS2,OBJC_ROWC,ROWVDEG,...,PSF_CLEAN_MJD_MAXDIFF_u,PSF_CLEAN_MJD_MAXDIFF_g,PSF_CLEAN_MJD_MAXDIFF_r,PSF_CLEAN_MJD_MAXDIFF_i,PSF_CLEAN_MJD_MAXDIFF_z,PSF_CLEAN_MJD_VAR_u,PSF_CLEAN_MJD_VAR_g,PSF_CLEAN_MJD_VAR_r,PSF_CLEAN_MJD_VAR_i,PSF_CLEAN_MJD_VAR_z
0,8162,b'301',4,80,23,6,-1845096432,47120,71.752632,-0.007592,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,8162,b'301',4,80,28,6,-1878126544,47120,83.333847,-0.002670,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,8162,b'301',4,80,31,6,268566528,36880,87.819733,0.000778,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,8162,b'301',4,80,33,6,268435456,16,97.791443,0.000968,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,8162,b'301',4,80,38,6,-1878650864,112656,203.664322,0.007192,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88656,8162,b'301',4,237,1416,6,268435712,65552,1350.132202,0.000000,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
88657,8162,b'301',4,237,1418,6,268436224,65552,1374.858887,0.004707,...,2,2,2,2,2,2.0,2.0,2.0,2.0,2.0
88658,8162,b'301',4,237,1420,6,-1879047920,24592,1374.583618,0.004490,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
88659,8162,b'301',4,237,1423,6,268435712,65552,1424.245361,0.000000,...,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
